In [1]:
import numpy as np
import pandas as pd
import json
from datetime import datetime
today = datetime.now().strftime('%d %B %Y')

In [7]:
# https://datacatalog.cookcountyil.gov/resource/wvhk-k5uv.csv?$select=pin, nbhd_code, class, sale_price&$where=year>2018 AND starts_with(class,'2')&$limit=1000000000000
sales = pd.read_csv('sales_2019+_major_class_2.csv')
sales['pin'] = sales['pin'].astype('str').str.zfill(14)
sales['nbhd_code'] = sales['nbhd_code'].astype('str')

/var/folders/7b/cxqkjv5514q19434nrk5rx0r0000gp/T/ipykernel_2011/798435012.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  sales = pd.read_csv('sales_2019+_major_class_2.csv')


In [2]:
# https://datacatalog.cookcountyil.gov/resource/uzyt-m557.csv?$select=pin, class, certified_tot, certified_land, certified_bldg&$where=year=2022 AND starts_with(class, '2') AND certified_tot > 0&$limit=1000000000000
assessed = pd.read_csv('assessed_2022_major_class_2_nonzero_certified_tot.csv')
assessed['pin'] = assessed['pin'].astype('str').str.zfill(14)

In [3]:
# https://datacatalog.cookcountyil.gov/resource/nj4t-kc8j.csv?$select=pin, class, triad_name, nbhd_code, ward_num, ward_chicago_data_year, chicago_community_area_name, chicago_community_area_data_year&$where=year=2022&$limit=1000000000000
loc = pd.read_csv('parcel_universe_2022.csv')
loc['pin'] = loc['pin'].astype('str').str.zfill(14)

In [4]:
def aggregate(source, groupby, columns, round_to_int = True, include_total = True, modify_index = None):
    multiple_cols = True if isinstance(columns, list) and len(columns) > 1 else False
    grouped = source.groupby(
        by=groupby
    )[columns].describe(
        percentiles = [.5]
    ).rename(
        columns = {'50%': 'median'}
    )
    if modify_index is not None:
        grouped.index = modify_index(grouped.index)
    
    if multiple_cols:
        tmp = pd.DataFrame(
            index = pd.MultiIndex.from_arrays([
                np.ravel([[i]*grouped.columns.levels[0].shape[0] for i in grouped.index]),
                list(grouped.columns.levels[0])*grouped.index.shape[0]
            ]),
            columns = grouped.columns.levels[1]
        )
        for i in grouped.index:
            for j in grouped.columns.levels[0]:
                tmp.loc[i, j] = grouped.loc[i, j]
        grouped = tmp

    if include_total:
        total = source[columns].describe(
            percentiles = [.5]
        ).rename(
            {'50%': 'median'}
        )
        if multiple_cols:
            total = total.T
            total.index = pd.MultiIndex.from_arrays([['total']*total.index.shape[0], total.index])
            total = total[grouped.columns].sort_index(level=0)
            grouped = pd.concat([grouped, total])
        else:
            grouped.loc['total'] = total
    return grouped.round(0).astype('int') if round_to_int else grouped

In [6]:
nbhd_list = pd.read_csv('neighborhood_list.csv')['nbhd_code'].astype('str').unique()

In [12]:
merged = pd.merge(
    sales,
    loc,
    how = 'left',
    on = 'pin',
    suffixes = ["", "_parcel_universe"]
)

sales_data = {
    'sales_by_ward' : aggregate(
        source = merged[~pd.isna(merged['ward_chicago_data_year'])],
        groupby = ['ward_num'],
        columns = 'sale_price',
        modify_index = lambda index: index.astype('int'),
    ),
    'sales_by_nbhd' : aggregate(
        source = merged[merged['nbhd_code'].isin(nbhd_list)],
        groupby = ['nbhd_code'],
        columns = 'sale_price'
    ),
    'sales_by_community' : aggregate(
        source = merged[~pd.isna(merged['chicago_community_area_name'])],
        groupby = ['chicago_community_area_name'],
        columns = 'sale_price'
    )
}

with open('sales_data.js', 'w') as f:
    f.write(f'const sales_date = "{today}";')
    for name, data in sales_data.items():
        f.write(f'\n\nconst {name} = ')
        f.write(data.to_json(orient='index'))
        f.write(';')

In [23]:
merged = pd.merge(
    assessed,
    loc,
    how = 'left',
    on = 'pin',
    suffixes = ["", "_parcel_universe"]
)
merged['nbhd_code'] = merged['nbhd_code'].astype('str')

assessed_data = {
    'assessed_by_ward' : aggregate(
        source = merged[~pd.isna(merged['ward_chicago_data_year'])],
        groupby = ['ward_num'],
        columns = ['certified_land', 'certified_bldg', 'certified_tot'],
        modify_index = lambda index: index.astype('int'),
    ),
    'assessed_by_nbhd' : aggregate(
        source = merged[merged['nbhd_code'].isin(nbhd_list)],
        groupby = ['nbhd_code'],
        columns = ['certified_land', 'certified_bldg', 'certified_tot'],
    ),
    'assessed_by_community' : aggregate(
        source = merged[~pd.isna(merged['chicago_community_area_name'])],
        groupby = ['chicago_community_area_name'],
        columns = ['certified_land', 'certified_bldg', 'certified_tot'],
    )
}

with open('assessed_data.js', 'w') as f:
    f.write(f'const assessed_date = "{today}";')
    for name, data in assessed_data.items():
        data = data.swaplevel().sort_index()
        f.write(f'\n\nconst {name} = ')
        f.write(json.dumps({ i: data.loc[i].to_dict(orient='index') for i in data.index.levels[0] }))
        f.write(';')